# Trend Screener

In [1]:
pip install "yfinance[nospam]" stocktrends matplotlib pandas numpy requests_cache


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#!/usr/bin/env pypy

import yfinance as yf
import pandas as pd
import numpy as np
# import requests_cache
from multiprocessing import Pool, cpu_count
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# session = requests_cache.CachedSession('yfinance.cache')
# session.headers['User-agent'] = 'Mozilla/5.0 (compatible; MSIE 7.0; Windows; U; Windows NT 10.4; Win64; x64; en-US Trident/4.0)'

results = []

tickers_txt_path = 'trend_screener_ticker.txt'

try:
    with open(tickers_txt_path, 'r') as file:
        tickers = [line.strip() for line in file if line.strip()]
except Exception as e:
    print(f"Error reading tickers from text file: {e}")
    tickers = []

start = '2020-03-02'
end = '2024-08-02'

data = yf.download(tickers, start = start, end = end)

print(data)

[*********************100%%**********************]  5552 of 5552 completed

3 Failed downloads:
['FLJ', 'AUVIP', 'AUVI']: YFChartError('%ticker%: No data found, symbol may be delisted')


Price        Adj Close                                                \
Ticker               A         AA  AACG   AACI  AACIU    AACT   AADI   
Date                                                                   
2020-03-02   80.872940  13.358201  0.98    NaN    NaN     NaN  9.615   
2020-03-03   79.245033  12.589594  0.98    NaN    NaN     NaN  9.585   
2020-03-04   83.221077  12.531219  1.00    NaN    NaN     NaN  9.945   
2020-03-05   79.383156  11.568027  0.96    NaN    NaN     NaN  9.900   
2020-03-06   77.508575  10.711859  1.00    NaN    NaN     NaN  9.525   
...                ...        ...   ...    ...    ...     ...    ...   
2024-07-26  137.320007  33.430000  0.65   9.55  11.09  10.720  1.580   
2024-07-29  137.380005  33.020000  0.65   8.76  11.09  10.740  1.590   
2024-07-30  139.490005  31.770000  0.65   9.11  10.99  10.730  1.560   
2024-07-31  141.399994  33.040001  0.66   8.29  10.99  10.750  1.550   
2024-08-01  142.860001  31.469999  0.65  10.45  10.99  10.745  1

In [4]:
def ehlers_instantaneous_trendline(source, alpha=0.07):
    itrend = np.zeros_like(source)
    trigger = np.zeros_like(source)

    for i in range(len(source)):
        if i < 7:
            itrend[i] = (source.iloc[i] + (2 * source.iloc[i - 1]) + source.iloc[i - 2]) / 4
        else:
            itrend[i] = (
                ((alpha - (alpha ** 2 / 4)) * source.iloc[i]) +
                (0.5 * alpha ** 2 * source.iloc[i - 1]) -
                ((alpha - (0.75 * alpha ** 2)) * source.iloc[i - 2]) +
                (2 * (1 - alpha) * itrend[i - 1]) -
                ((1 - alpha) ** 2 * itrend[i - 2])
            )

        if i >= 2:
            trigger[i] = (2 * itrend[i]) - itrend[i - 2]

    return itrend

def calculate_trend(df):
    df['52W_High'] = df['High'][-317:].rolling(window=252).max()
    df['52W_High_Ratio'] = df['High'][-1:] / df['52W_High'][-1:]
    df['52W_ROC'] = (df['High'][-317:] - df['High'].shift(252)) / df['High'].shift(252)
    
    df['52W_Low'] = df['Low'][-252:].rolling(window=252).min()
    df['52W_Low_Ratio'] = df['Low'][-1:] / df['52W_Low'][-1:]


    df['26W_High'] = df['High'].rolling(window=126).max()
    df['26W_High_Ratio'] = df['High'] / df['26W_High']
    df['26W_ROC'] = (df['High'][-146:] - df['High'].shift(126)) / df['High'].shift(252)

    df['50MA'] = df['Adj Close'][-51:].rolling(window=50).mean()
    df['150MA'] = df['Adj Close'][-151:].rolling(window=150).mean()
    df['200MA'] = df['Adj Close'][-282:].rolling(window=200).mean()
    df['250MA'] = df['Adj Close'][-251:].rolling(window=250).mean()
    df['750MA'] = df['Adj Close'][-756:].rolling(window=750).mean()
    df['1000MA'] = df['Adj Close'][-1006:].rolling(window=1000).mean()

    df['200MA_Uptrend'] = df['200MA'][-81:].diff() > 0
    df['200MA_1M_Uptrend'] = df['200MA_Uptrend'][-20:].rolling(window=20).mean() * 100
    df['200MA_4M_Uptrend'] = df['200MA_Uptrend'][-80:].rolling(window=80).mean() * 100

    itrend = ehlers_instantaneous_trendline(df['Adj Close'])
    df['ITrend'] = itrend
    df['ITrend_Uptrend'] = df['ITrend'][-2:].diff() > 0
    
    return df

def process_trend(ticker):
    try:
        print(f"process trend {ticker}")
        ticker_df = pd.DataFrame(data['Adj Close'][ticker])
        ticker_df['High'] = data['High'][ticker]
        ticker_df['Low'] = data['Low'][ticker]
        ticker_df = ticker_df.rename(columns={ticker: 'Adj Close'})
        ticker_df = calculate_trend(ticker_df)

        return ticker, ticker_df
    except Exception as e:
        print(f"Error calculating trend for {ticker}: {e}")
        return ticker, None

results = {}

with Pool(processes=cpu_count()) as pool:
    trend_results = pool.map(process_trend, tickers)

for ticker, result in trend_results:
    if result is not None:
        results[ticker] = result

roc52w_data = pd.DataFrame({ticker: df['52W_ROC'] for ticker, df in results.items()})
roc52w_ranks = roc52w_data.rank(axis=1)
roc52w_ranks = roc52w_ranks.div(len(roc52w_ranks.columns))

roc26w_data = pd.DataFrame({ticker: df['26W_ROC'] for ticker, df in results.items()})
roc26w_ranks = roc26w_data.rank(axis=1)
roc26w_ranks = roc26w_ranks.div(len(roc26w_ranks.columns))

def process_rs(ticker):
    try:
        print(f"process rs {ticker}")
        results[ticker]['RS52W'] = roc52w_ranks[ticker]
        results[ticker]['RS52W_13W'] = results[ticker]['RS52W'].rolling(window=65).mean()

        results[ticker]['RS26W'] = roc26w_ranks[ticker]
        results[ticker]['RS26W_4W'] = results[ticker]['RS26W'].rolling(window=20).mean()
    except Exception as e:
        print(f"Error adding RS for {ticker}: {e}")

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(process_rs, ticker): ticker for ticker in tickers}
    for future in as_completed(futures):
        future.result()


process trend AXSprocess trend AQMSprocess trend Aprocess trend AIRT
process trend BIPI
process trend BWAY


process trend CHK

process trend COST
process trend BWB
process trend AQN
process trend AXSM
process trend BIRDprocess trend AIRTP

process trend CHKELprocess trend AA

process trend COTY
process trend BITprocess trend BWBBP
process trend AXTA

process trend AACG
process trend AQNB
process trend AIT
process trend CHKEW
process trend COUR
process trend AXTIprocess trend AACI

process trend AQST
process trend BWFG
process trend BITF
process trend AIU
process trend CHKEZ
process trend COYA
process trend AACIU
process trend AY
process trend AQU
process trend BWG
process trend CHKPprocess trend BIVI

process trend AIV
process trend CP
process trend AACT
process trend AYI
process trend AR
process trend BJprocess trend AIXI

process trend CHMGprocess trend BWMN

process trend AADI
process trend AYRO
process trend BJDXprocess trend CPA

process trend ARAY
process trend BWMXprocess trend

In [5]:
summary_data = {}
for ticker, df in results.items():
    latest_data = df.iloc[-1]
    g = (df.iloc[-1]['200MA'] / df.iloc[-2]['200MA'] - 1) if df.iloc[-2]['200MA'] != 0 else 0
    g_1000 = (df.iloc[-1]['1000MA'] / df.iloc[-2]['1000MA'] - 1) if df.iloc[-2]['1000MA'] != 0 else 0
    stop_loss = 1 - latest_data['50MA'] / latest_data['Adj Close']
    ma_uptrend =     df['MA_Uptrend'] = (
        (latest_data['Adj Close'] > latest_data['150MA']) & 
        # (df['50MA'][-2:] > df['150MA'][-2:]) &
        (latest_data['150MA'] > latest_data['200MA']) &
        (latest_data['200MA'] > latest_data['250MA']) &
        (latest_data['250MA'] > latest_data['750MA']) &
        (latest_data['750MA'] > latest_data['1000MA']) &
        (df.iloc[-1]['50MA'] > df.iloc[-2]['50MA']) &
        (df.iloc[-1]['150MA'] > df.iloc[-2]['150MA']) &
        (df.iloc[-1]['200MA'] > df.iloc[-2]['200MA']) &
        (df.iloc[-1]['250MA'] > df.iloc[-2]['250MA']) &
        (df.iloc[-1]['750MA'] > df.iloc[-2]['750MA']) &
        (df.iloc[-1]['1000MA'] > df.iloc[-2]['1000MA'])
    )


    risk_reward = (g / stop_loss) if stop_loss != 0 else 0
    buy_signal = latest_data['52W_High_Ratio'] >= 0.75 and \
        latest_data['52W_Low_Ratio'] >= 1.25 and \
        latest_data['52W_ROC'] >= 0.25 and \
        latest_data['26W_ROC'] >= 0.15 and \
        ma_uptrend and \
        latest_data['ITrend_Uptrend'] and \
        latest_data['RS26W'] >= 0.7 and \
        latest_data['RS26W_4W'] >= 0.6 and \
        latest_data['RS52W'] >= 0.7 and \
        latest_data['RS52W_13W'] >= 0.6 and \
        stop_loss >= 0.02 and \
        stop_loss <= 0.15 and \
        g * 100 > 0.2 and \
        g_1000 * 100 >= 0.15 and \
        risk_reward * 10 >= 0.2


    # if uptrend:
    summary_data[ticker] = {
        'Ticker': ticker,
        '52W_High_Ratio': round(latest_data['52W_High_Ratio'], 2),
        '52W_ROC': round(latest_data['52W_ROC'], 2),
        '26W_High_Ratio': round(latest_data['26W_High_Ratio'], 2),
        '26W_ROC': round(latest_data['26W_ROC'], 2),
        'RS52W': round(latest_data['RS52W'], 2),
        'RS52W_13W': round(latest_data['RS52W_13W'], 2),
        'RS26W': round(latest_data['RS26W'], 2),
        'RS26W_4W': round(latest_data['RS26W_4W'], 2),
        'Trend_Ratio': round(latest_data['ITrend']/ latest_data['200MA'] * 100, 2),
        'Stop_Loss_Pct': round(stop_loss * 100, 2),
        'g_Daily_Pct': round(g * 100, 4),
        'g_1000_Daily_Pct': round(g_1000 * 100, 4),
        'Risk_Reward_10D': round(risk_reward * 10, 2),
        'MA_Uptrend': ma_uptrend,
        'ITrend_Uptrend': latest_data['ITrend_Uptrend'],
        'Buy_Signal': buy_signal
    }

summary_df = pd.DataFrame(summary_data).T
summary_df = summary_df.sort_values(by=['Buy_Signal', 'Trend_Ratio', 'Risk_Reward_10D', 'g_Daily_Pct', 'RS26W'], ascending=[False, True, False, False, False])

summary_df.to_csv('filtered_stocks.csv', index=False)

print(summary_df)

     Ticker 52W_High_Ratio 52W_ROC 26W_High_Ratio 26W_ROC RS52W RS52W_13W  \
XPO     XPO           0.94    0.74           0.94    0.48  0.94      0.95   
IOR     IOR           0.97    0.57           0.97    0.38  0.91      0.87   
KTB     KTB           0.98    0.72           0.98    0.29  0.94      0.92   
GFF     GFF           0.94     0.7           0.94    0.29  0.93      0.94   
HCA     HCA            1.0    0.33            1.0    0.22  0.83       0.7   
...     ...            ...     ...            ...     ...   ...       ...   
QDRO   QDRO            NaN     NaN            NaN     NaN   NaN       NaN   
SBOW   SBOW            NaN     NaN            NaN     NaN   NaN       NaN   
SDPI   SDPI            NaN     NaN            NaN     NaN   NaN       NaN   
SEPA   SEPA            NaN     NaN            NaN     NaN   NaN       NaN   
SLCA   SLCA            NaN     NaN            NaN     NaN   NaN       NaN   

     RS26W RS26W_4W Trend_Ratio Stop_Loss_Pct g_Daily_Pct g_1000_Daily_Pct 